In [112]:
# Import necessary libraries
import sqlite3
import pandas as pd


# Connect to a SQLite database (or create one)
conn = sqlite3.connect(r'C:\Users\spiri\codingbootcamp\Final-project-C4Y\data.db')
cursor = conn.cursor()

WITH mean_deposite as (
    select client_id, AVG(balance), currency
    from balances
    group by 1
)
Select * 
from inv_campaign_eval
left join client
on inv_campaign_eval.client_id = client.client_id
left join client_products
on inv_campaign_eval.client_id = client_products.client_id
left join mean_deposite
on inv_campaign_eval.client_id = mean_deposite.client_id

In [113]:
query = ('''
WITH mean_deposite as (
    select client_id, AVG(balance) as mean_balance, currency
    from balances
    group by 1
)
Select * 
from inv_campaign_eval
left join client
on inv_campaign_eval.client_id = client.client_id
left join client_products
on inv_campaign_eval.client_id = client_products.client_id
left join mean_deposite
on inv_campaign_eval.client_id = mean_deposite.client_id''')
df = pd.read_sql_query(query, conn)

df.head()

,client_id,poutcome,client_id,age,job,marital,education,gender,client_id,has_deposits,loan,has_insurance,has_mortgage,client_id,mean_balance,currency
0,249789938,success,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,249789938,2669.532734,CZK
1,1504633819,failure,1504633819,38.0,management,married,tertiary,M,1504633819,no,yes,n,yes,1504633819,5440.862334,CZK
2,1952195738,success,1952195738,34.0,services,married,secondary,M,1952195738,yes,yes,n,no,1952195738,2922.958852,CZK
3,989726384,success,989726384,62.0,retired,married,secondary,F,989726384,yes,no,n,no,989726384,3124.614843,CZK
4,1971878545,failure,1971878545,36.0,management,single,tertiary,M,1971878545,yes,no,n,no,1971878545,474.083033,CZK


In [29]:
print(df.columns)
print(df.dtypes)


Index(['client_id', 'poutcome', 'age', 'job', 'marital', 'education', 'gender',
       'has_deposits', 'loan', 'has_insurance', 'has_mortgage', 'mean_balance',
       'currency'],
      dtype='object')
client_id        object
poutcome         object
age              object
job              object
marital          object
education        object
gender           object
has_deposits     object
loan             object
has_insurance    object
has_mortgage     object
mean_balance     object
currency         object
dtype: object


Here we drop duplicate columns

In [114]:
df = df.T.drop_duplicates().T

In [79]:
df.head()

,client_id,poutcome,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,mean_balance,currency
0,249789938,success,38.0,services,married,secondary,M,yes,no,n,no,2669.532734,CZK
1,1504633819,failure,38.0,management,married,tertiary,M,no,yes,n,yes,5440.862334,CZK
2,1952195738,success,34.0,services,married,secondary,M,yes,yes,n,no,2922.958852,CZK
3,989726384,success,62.0,retired,married,secondary,F,yes,no,n,no,3124.614843,CZK
4,1971878545,failure,36.0,management,single,tertiary,M,yes,no,n,no,474.083033,CZK


checking and dealing with the missing value

In [80]:
df.isna().sum()

client_id         0
poutcome          0
age              50
job              83
marital           0
education         0
gender            0
has_deposits      0
loan              0
has_insurance     0
has_mortgage      0
mean_balance      0
currency          0
dtype: int64

In [115]:
df.job.fillna(value='unknown', inplace=True)

C:\Users\spiri\AppData\Local\Temp\ipykernel_12160\1783974414.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.job.fillna(value='unknown', inplace=True)


In [116]:
df.age.fillna(value=df.age.mean(), inplace=True)


C:\Users\spiri\AppData\Local\Temp\ipykernel_12160\2420278621.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.age.fillna(value=df.age.mean(), inplace=True)
C:\Users\spiri\AppData\Local\Temp\ipykernel_12160\2420278621.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.age.fillna(value=df.age.mean(), in

So we fill the missing values, no we can focus on the data types

In [124]:
df.head()

,client_id,poutcome,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,mean_balance,currency
0,249789938,1,38.0,services,1,2,1,1,0,0,0,2669.532734,CZK
1,1504633819,0,38.0,management,1,3,1,0,1,0,1,5440.862334,CZK
2,1952195738,1,34.0,services,1,2,1,1,1,0,0,2922.958852,CZK
3,989726384,1,62.0,retired,1,2,0,1,0,0,0,3124.614843,CZK
4,1971878545,0,36.0,management,2,3,1,1,0,0,0,474.083033,CZK


In [118]:
df.mean_balance = pd.to_numeric(df.mean_balance)

In [119]:
df.client_id = pd.to_numeric(df.client_id)

In [120]:
df.mean_balance.sum()

3975208.4593620505

next part I want to focus on converting currency everyrhing to czk


In [121]:
rate = {'CZK': 1, 'USD': 23, 'EUR': 25}

df['mean_balance'] = df.apply(lambda row: row['mean_balance'] * rate[row['currency']], axis=1)

In [123]:
df.job.unique()

array(['services', 'management', 'retired', 'admin.', 'student',
       'entrepreneur', 'blue-collar', 'technician', 'unknown',
       'housemaid', 'self-employed', 'unemployed'], dtype=object)

So here we can slowly start an feature engeniring

In [122]:
from sklearn.preprocessing import LabelEncoder
education_map = {'primary':1, 'secondary':2, 'tertiary':3,'unknown':0}
df.education = df.education.replace(education_map)

label_encoders = {}
for col in ['marital', 'has_deposits', 'poutcome', 'loan', "has_insurance", 'has_mortgage', 'gender']:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

C:\Users\spiri\AppData\Local\Temp\ipykernel_12160\342006815.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.education = df.education.replace(education_map)


In [125]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe_result = ohe.fit_transform(df[['job', 'currency']])
ohe_df = pd.DataFrame(ohe_result.toarray(), columns=ohe.get_feature_names_out(['job', 'currency']))
df = pd.concat([df, ohe_df], axis=1)



In [127]:
df.drop(columns=['job', 'currency'])

,client_id,poutcome,age,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,...,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,currency_CZK,currency_EUR,currency_USD
0,249789938,1,38.0,1,2,1,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1504633819,0,38.0,1,3,1,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1952195738,1,34.0,1,2,1,1,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,989726384,1,62.0,1,2,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1971878545,0,36.0,2,3,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2294,1270855655,0,58.0,0,2,0,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2295,471724023,1,72.0,1,1,1,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2296,620088171,0,34.0,1,3,0,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2297,1518319265,1,39.0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [126]:
df.head()

,client_id,poutcome,age,job,marital,education,gender,has_deposits,loan,has_insurance,...,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,currency_CZK,currency_EUR,currency_USD
0,249789938,1,38.0,services,1,2,1,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1504633819,0,38.0,management,1,3,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1952195738,1,34.0,services,1,2,1,1,1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,989726384,1,62.0,retired,1,2,0,1,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1971878545,0,36.0,management,2,3,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
